# Discovering Sentinel-1 SLC data via SARA <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Compatability:** Notebook currently compatible with the `DEA Sandbox` environment only
* **Products used:** 
[S1 SLC granules](https://copernicus.nci.org.au/sara.client/#/explore?collection=S1)
* **Special requirements:** Need to `pip install` the `auscophub` and `PyRAT` packages directly from Github

## Background
This notebook demonstrates a recipe for working with Sentinel-1 Single Look Complex (SLC) data to quickly identify possible burn scar areas using Complex data at the native resolution of the SAR sensor. This notebook was developed during a SAR Workshop day and will be progressed into a Real World Sample as actual fire based variability is noted. The sample area used in this notebook is over [Orroral Valley](https://www.google.com/maps/place/Orroral+Valley/@-35.6168882,148.6665058,10z/data=!4m13!1m7!3m6!1s0x6b17d73435882e61:0x2440e61df2404dce!2sOrroral+Valley!3b1!8m2!3d-35.61689!4d148.9466573!3m4!1s0x6b17d73435882e61:0x2440e61df2404dce!8m2!3d-35.61689!4d148.9466573).

The [Sentinel-1 mission](https://sentinel.esa.int/web/sentinel/missions/sentinel-1) comprises a constellation of two polar-orbiting satellites, operating day and night performing C-band synthetic aperture radar imaging, enabling them to acquire imagery regardless of the weather. The Single Look Complex Data product represents the [first level of processing](https://sentinel.esa.int/web/sentinel/level-1-slc-processing-algorithms) beyond raw echoes at the sensor level to form an image.

[SARA](http://www.copernicus.gov.au/regionaldataaccess) provides intuitive map-based data search and download capability, as well as an API for advanced user interaction. This notebook uses the REST API via a purpose built Python client to search and download imagery.

## Description
This notebook sets up some basic steps to find and work with SLC data:

1. Install [Auscophub](https://github.com/CopernicusAustralasia/auscophub) Python package. 
1. Locate Sentinel-1 SLC datasets over Orroral Valley using [Copernicus Hub Australia](https://copernicus.nci.org.au/sara.client/#/explore).
2. Download the SARA results and unpack them.
3. Perform some exploratory data extraction and plotting using GDAL [Complex Tiff](https://gdal.org/drivers/raster/gtiff.html) with `Cfloat64` datatype, and allocate enough RAM for this.
4. Install the [PyRAT](https://github.com/birgander2/PyRAT) Python package.
5. Load Sentinel-1 SLC data for a specific granule and produce a preview.

***

## Getting started

This notebook requires two packages which should be installed via `!pip install --user` in a notebook cell or in the console.

### Load packages

Load various packages and install additional ones in `--user` mode:

In [1]:
!export DISPLAY=0:0 && pip install --user git+https://github.com/whatnick/PyRAT
!pip install --user git+https://github.com/CopernicusAustralasia/auscophub

ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.


In [2]:
import os
import tqdm
import json
import glob
import datacube
import requests
import zipfile
import logging
import numpy as np
import osgeo.gdal as gdal
from datetime import datetime
from auscophub import saraclient
from shapely.geometry import Point
# from pylab import plot, imshow, figure, scatter
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'auscophub'

## Load data from SARA

### Connect to SARA and get some search results over Orroral
- Create a 0.01 degree buffer around the point of interest
- Perform a parametric search query and derive an array of results

In [ ]:
# Set location for SARA query
orroral_polygon = Point(148.96,-35.58).buffer(0.01)

queryParams = [
               'collection=S1',
               'instrument=C-SAR',
               'productType=SLC',
               'startDate=2020-01-01',
               'completionDate=2020-02-04',
               'page=1',
               'orbitNumber=147',
               f'geometry={orroral_polygon.to_wkt()}']

# Run SARA query
urlOpener = saraclient.makeUrlOpener()
results = saraclient.searchSara(urlOpener, 1, queryParams)

# Extract scene ideas from the query result
scene_ids = [x['properties']['productIdentifier'] for x in results]

# Print scene IDs
print(scene_ids)

### Download SARA results using Authenticated Access
* [Sign up](https://copernicus.nci.org.au/sara.client/#/register) to SARA and note down the username/password for your account.
* Create a file called **sara_creds.json**
* Store username and password in this file in this format:
    ```
    {
        "username":"username@ga.gov.au",
        "password":"download_password"
    }
    ```
* Read this JSON into the notebook and use it authenticate to SARA and download the data
* This notebook includes a convenience download method with a progress bar. **This step can take several minutes to complete.**

In [ ]:
def download_s1_file(result, verbose = False, auth = None):
    """
    Download file with progressbar
    Uses adapted version of the code here :
    https://gist.github.com/ruxi/5d6803c116ec1130d484a4ab8c00c603
    """
    local_filename = f"{result['properties']['productIdentifier']}.zip"
    local_filename = os.path.join('/notebooks',local_filename)
    url = result['properties']['services']['download']['url']
    r = requests.get(url, stream=True, auth = auth)
    file_size = int(r.headers['Content-Length'])
    chunk = 1
    chunk_size=1024
    num_bars = int(file_size / chunk_size)
    if verbose:
        print(dict(file_size=file_size))
        print(dict(num_bars=num_bars))

    with open(local_filename, 'wb') as fp:
        for chunk in tqdm.tqdm(r.iter_content(chunk_size=chunk_size), 
                               total= num_bars,
                               unit = 'KB',
                               desc = local_filename,
                               leave = True # progressbar stays
                               ):
            fp.write(chunk)
            fp.flush()
    return

# Load credentials and download data
credentials = json.load(open('sara_creds.json'))
auth=(credentials['username'], credentials['password'])
for result in results:
    scene_id = result['properties']['productIdentifier']
    print(scene_id)
    filename = f'{scene_id}.zip'
    filename = os.path.join('/notebooks', filename)
    if not os.path.exists(filename):
        try:
            download_s1_file(result, auth=auth)
        except HTTPError as hte:
            print(f"Failed to Download: {url}")

### Extract data
Please be patient: **this step can take several minutes to complete**.

In [ ]:
time_start = datetime.now()
for result in results:
    scene_id = result['properties']['productIdentifier']
    filename = os.path.join('/notebooks', f'{scene_id}.zip')
    out_dir = f'{scene_id}.SAFE'
    out_dir = os.path.join('/notebooks', out_dir)
    print(f'Extracting data from {filename}')
    if not os.path.isdir(out_dir):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            try:
                zip_ref.extractall('/notebooks')
            except:
                print(f"    Corrupted: {filename}")
                os.unlink(filename)
time_finish = datetime.now()
print(f"Extracted in {str(time_finish-time_start)}")

## Analyse SLC data

### GDAL GCP Mechanism for quick georeferencing
- SLC data carries a grid of GCPs for indicative georeferencing
- [Helmert transforms](https://proj.org/operations/transformations/helmert.html) can be performed for low distortion warping of SLC domain products approximately to geographic co-ordinates

In [ ]:
s1_ds = gdal.Open('/notebooks/S1A_IW_SLC__1SDV_20200129T191613_20200129T191639_031019_039028_1537.SAFE')

sds_list = s1_ds.GetSubDatasets()
IW1_ds = gdal.Open(sds_list[2][0])
IW2_ds = gdal.Open(sds_list[5][0])
IW3_ds = gdal.Open(sds_list[8][0])

lon_1 = [gcp.GCPX for gcp in IW1_ds.GetGCPs()]
lat_1 = [gcp.GCPY for gcp in IW1_ds.GetGCPs()]
lon_2 = [gcp.GCPX for gcp in IW2_ds.GetGCPs()]
lat_2 = [gcp.GCPY for gcp in IW2_ds.GetGCPs()]
lon_3 = [gcp.GCPX for gcp in IW3_ds.GetGCPs()]
lat_3 = [gcp.GCPY for gcp in IW3_ds.GetGCPs()]

fig, axs = plt.subplots(nrows=1, ncols=1, constrained_layout=True)
axs.scatter(lon_1, lat_1, color='k')
axs.scatter(lon_2, lat_2, color='b')
axs.scatter(lon_2, lat_2, color='g')


### Some exploratory plotting of Orroral SLC data

The exploratory plots show very good alignment of the data in both dynamic range and geographic extents purely in pixel domain without additional geometric and radiometric corrections due to uniform illumination and look-angle provided by the SAR sensor on repeat orbits. As such the SLC data can be used for relative analysis and burnt area mapping without expensive post-processing.

* Plot image snippets to check alignment in pixel co-ordinates
* Plot histograms to show any overall backscatter changes

In [ ]:
datasets = [f'{scene}.SAFE/measurement' for scene in scene_ids]
data_collection = {}
for data in datasets:
    tiff_files = glob.glob(os.path.join('/notebooks',data,'*.tiff'))
    for tiff in tiff_files:
        if 's1a-iw2' in tiff:
            ds = gdal.Open(tiff)
            xsize = ds.RasterXSize
            ysize = ds.RasterYSize
            data = ds.ReadAsArray(int(xsize/4),int(ysize/4),1000,1000)
            data_collection[tiff] = data

In [ ]:
fig, axs = plt.subplots(nrows=2,
                        ncols=int(len(data_collection) / 2),
                        constrained_layout=True)
fig.set_size_inches(12, 8, forward=True)
i = 0
keys = list(data_collection.keys())
for ax in axs.flat:
    key = keys[i]
    decibel = np.log(np.absolute(data_collection[keys[i]])**2) * 10
    ax.imshow(decibel)
    basename = os.path.basename(key)
    ax.set_title(basename[12:23])
    i += 1


In [ ]:
fig, axs = plt.subplots(nrows=int(len(data_collection) / 2),
                        ncols=2,
                        constrained_layout=True)
fig.set_size_inches(10.5, 18.5, forward=True)
i = 0
keys = list(data_collection.keys())
for ax in axs.flat:
    key = keys[i]
    basename = os.path.basename(key)
    pol_date = basename[12:23]
    data = data_collection[keys[i]]
    data_no_nan = np.nan_to_num(np.absolute(data))
    counts, bins = np.histogram(data_no_nan.flatten(), bins=100)
    ax.hist(bins[:-1], bins=100, weights=counts)
    ax.set_title(pol_date)
    i += 1


### Load and manipulate data in PyRAT
[PyRat](https://github.com/birgander2/PyRAT/wiki/About-PyRAT) (Python Radar Tools) is a flexible open-source framework for post-processing synthetic aperture radar (SAR) data. PyRAT is implementedin Python and distributed under an open-source license. It is mostly intended for post-processing of both airborne and spaceborne SAR imagery. PyRAT features an easy to use plugin-based programming interface, which allows users to quickly extend it with their own routines.

This section of the notebook demostrate headless/scripted use of PyRAT on the SLC data downloaded previously.

* Load data from one SLC scene into PyRAT
* The load generates two layers
* Perform some filtering and preview generation

In [ ]:
os.environ['DISPLAY']='0:0'
from pyrat import *
pyrat_init()

In [ ]:
s1 = load.sentinel1(dir='/notebooks/S1A_IW_SLC__1SDV_20200129T191613_20200129T191639_031019_039028_1537.SAFE',swath=2)

In [ ]:
info()

In [ ]:
activate(['/L1','/L2'])

In [ ]:
getmeta()

In [ ]:
s1_l1_box = filter.boxcar(layer=s1[0])
save.pixmap('s1_l1.jpg',layer=s1[0],scaling=10)
save.pixmap('s1_l1_box.jpg', layer=s1_l1_box,scaling=10)

***

## Future work

**Burn Scar Mapping**: Fire Mapping in SAR Reference : https://www.mdpi.com/2072-4292/9/8/764/htm

**Entropy/Alpha and Statistical Analysis**: Analyse them in Slant Range with [PyRAT](https://github.com/birgander2/PyRAT)

**Create Near-Real Time Results**: Use [Helmert Transforms](https://en.wikipedia.org/wiki/Helmert_transformation) to utilize GCP's embedded in metadata to go from pixel domain/SLC domain to real-world coordinates.

**Create Accurate Results**: Convert analysis results to Ground range/Orthorectified versions using [SNAP](http://step.esa.int/main/download/snap-download/)/[Gamma](https://www.gamma-rs.ch/)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** February 2020

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)